In [ ]:
# Instalação do pacote CrewAI e suas ferramentas via pip
pip install crewai crewai-tools

In [ ]:
# Atualização do pacote CrewAI e suas ferramentas para a versão mais recente via pip
pip install --upgrade crewai crewai-tools

In [ ]:
# Verifica se o pacote CrewAI está instalado e exibe sua versão
pip freeze | grep crewai

crewai==0.80.0
crewai-tools==0.14.0


In [ ]:
# Cria um novo projeto CrewAI
!crewai create crew LinkedIN-demo

Creating folder linkedin_demo...
Select a provider to set up:
1. openai
2. anthropic
3. gemini
4. groq
5. ollama
6. watson
7. bedrock
8. azure
9. cerebras
10. other
q. Quit
Enter the number of your choice or 'q' to quit: 4
Select a model to use for Groq:
1. groq/llama-3.1-8b-instant
2. groq/llama-3.1-70b-versatile
3. groq/llama-3.1-405b-reasoning
4. groq/gemma2-9b-it
5. groq/gemma-7b-it
q. Quit
Enter the number of your choice or 'q' to quit: 1
Enter your GROQ API key (press Enter to skip): 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
API keys and model saved to .env file
Selected model: groq/llama-3.1-8b-instant
  - Created linkedin_demo/.gitignore
  - Created linkedin_demo/pyproject.toml
  - Created linkedin_demo/README.md
  - Created linkedin_demo/src/linkedin_demo/__init__.py
  - Created linkedin_demo/src/linkedin_demo/main.py
  - Created linkedin_demo/src/linkedin_demo/crew.py
  - Created linkedin_demo/src/linkedin_demo/tools/custom_tool.py
  - Created linkedin_demo/src

In [40]:
# Importa as classes principais do pacote CrewAI
# Agent: representa um agente que pode realizar tarefas
# Task: representa uma tarefa que pode ser realizada por um agente
# Crew: representa um grupo de agentes que podem trabalhar juntos
from crewai import Agent, Task, Crew

In [41]:
# Configura as variáveis de ambiente para o uso de APIs externas
# Importa o módulo os para manipular variáveis de ambiente
import os

# Importa o módulo userdata do Google Colab para acessar dados do usuário
from google.colab import userdata

# Define a chave da API do OpenAI
# A chave é obtida do usuário via userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Define o nome do modelo do OpenAI a ser usado
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4'

# Define a chave da API do Serper
# A chave é obtida do usuário via userdata
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')


In [42]:
# Importa módulos e classes para realizar buscas e raspagem de dados
# Importa o módulo search do pacote re para realizar buscas regulares
from re import search

# Importa classes de ferramentas do pacote crewai_tools
# SerperDevTool: ferramenta para realizar buscas no Serper
# ScrapeWebsiteTool: ferramenta para realizar raspagem de dados em websites
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Instancia a ferramenta de busca do Serper
search_tool = SerperDevTool()

# Instancia a ferramenta de raspagem de dados em websites
scrape_tool = ScrapeWebsiteTool()

# Fluxo:
1 - Buscador de Conteúdo
2 - Redator de Conteúdo
3 - Editor de Conteúdo


Agente 1 - Buscador de Conteúdo

In [53]:
buscador = Agent(

                 role='Buscador de Conteúdo',
                 goal='Realizar uma busca exaustiva e eficiente de conteúdo online sobre {tema}',
                 backstory='Você é um especialista em busca de conteúdo online, '
                                'trabalhando na criação de artigos para o LinkedIN sobre {tema}. '
                                'Seu objetivo é encontrar as informações mais relevantes e atualizadas '
                                'sobre o tema, coletar e organizar essas informações de forma eficiente, '
                                'para que o Redator de Conteúdo possa criar artigos de alta qualidade.',
                 tools=[search_tool, scrape_tool],
                 verbose=True
)

Agente 2 - Redator de Conteúdo

In [54]:
redator = Agent(
                 role='Redator de Conteúdo',
                 goal='Criar um conteúdo atraente e informativo para o LinkedIn sobre {tema}, utilizando emojis e destacando os principais pontos técnicos e potenciais.',
                backstory='Você é um redator de conteúdo especializado em criar artigos para o LinkedIn. '
                            'Seu objetivo é criar um texto que seja atraente e informativo para o público-alvo, '
                            'utilizando os dados coletados pelo Buscador de Conteúdo.',
                 tools=[search_tool, scrape_tool],
                 verbose=True
)

Agente 3 - Editor de Conteúdo

In [55]:
editor = Agent(
                 role='Editar de Conteúdo',
                 goal='Editar e revisar um texto de LinkedIn sobre {tema}, adicionando emojis e bullets para torná-lo mais atraente e fácil de ler.',
                 backstory='Você é um especialista em edição de conteúdo, trabalhando na revisão de artigos para o LinkedIN.'
                          'Você vai receber um texto do Redator de Conteúdo e revisá-lo para garantir que esteja claro, conciso e atraente para o público-alvo.'
                          'Seu objetivo é criar um texto que seja fácil de ler e entender, e que inclua elementos visuais como emojis e bullets para torná-lo mais interessante.',
                 tools=[search_tool, scrape_tool],
                 verbose=True
)

# Tarefas

- Tarefa 1: Buscar

In [56]:
# Cria uma tarefa para o agente buscador de conteúdo
# A tarefa é buscar e priorizar as últimas tendências sobre um tema específico
buscar = Task(

    # Descrição da tarefa
    description=(
        '1. Identifique e priorize as últimas tendências e notícias sobre o {tema}, '
        'considerando a relevância e a atualidade das informações.\n'
        '2. Defina o público-alvo, considerando seus interesses, necessidades e pontos de dor, '
        'para garantir que o conteúdo seja relevante e atraente para eles.\n'
        '3. Inclua palavras-chave de SEO relevantes e fontes confiáveis, '
        'para garantir que o conteúdo seja encontrado e considerado autorizado.'
    ),

    # Agente responsável pela tarefa
    agent=buscador,

    # Saída esperada da tarefa
    expected_output='Um plano de tendências sobre {tema}, com as palavras-chave mais relevantes de SEO, '
                     'as últimas notícias e informações atualizadas, '
                     'além de uma definição clara do público-alvo e suas necessidades.'
)

- Redator

In [57]:
# Cria uma tarefa para o agente redator de conteúdo
# A tarefa é criar um post de LinkedIn sobre um tema específico
redigir = Task(

    # Descrição da tarefa
    description=(
        '1. Utilize os dados coletados para criar um post de LinkedIn atraente e informativo sobre o {tema}, '
        'considerando as tendências e necessidades do público-alvo.\n'
        '2. Incorpore palavras-chave de SEO de forma natural e estratégica, '
        'para garantir que o post seja encontrado e considerado relevante.\n'
        '3. Certifique-se de que o post seja interessante e envolvente, '
        'utilizando elementos visuais e linguagem acessível.\n'
        '4. Inicie o post com uma introdução que mencione que o conteúdo foi criado utilizando o {tema}, '
        'para estabelecer a credibilidade e autoridade do autor.\n'
        '5. Destaque os potenciais e pontos técnicos do {tema}, '
        'para fornecer valor agregado e insights relevantes para o público-alvo.'
    ),

    # Agente responsável pela tarefa
    agent=redator,

    # Saída esperada da tarefa
    expected_output='Um texto de LinkedIn sobre {tema}, que seja atraente, informativo e relevante, '
                     'com palavras-chave de SEO incorporadas de forma natural, '
                     'e que destaque os potenciais e pontos técnicos do tema.'
)

- Editor

In [58]:
# Cria uma tarefa para o agente editor de conteúdo
# A tarefa é revisar e editar a postagem do LinkedIn para garantir que esteja pronta para publicação
editar = Task(

    # Descrição da tarefa
    description=(
        'Revisar a postagem do LinkedIn para garantir que esteja livre de erros gramaticais, '
        'ortográficos e de pontuação, e que seja clara e concisa.\n'
        'Incluir emojis e bullets para tornar o texto mais atraente e fácil de ler.\n'
        'Verificar a consistência do estilo e do tom do texto, '
        'para garantir que seja coerente com a marca e o público-alvo.'
    ),

    # Agente responsável pela tarefa
    agent=editor,

    # Saída esperada da tarefa
    expected_output='Um texto de LinkedIn pronto para publicação, '
                     'com erros gramaticais e ortográficos corrigidos, '
                     'e que inclua emojis e bullets para tornar o texto mais atraente e fácil de ler. '
                     'O texto deve estar formatado com parágrafos e bullet points, '
                     'para garantir que seja fácil de ler e entender.'
)

# Cria uma equipe de agentes para realizar as tarefas de busca, redação e edição de conteúdo

In [59]:
equipe = Crew(
    # Lista de agentes que compõem a equipe
    agents=[buscador, redator, editor],

    # Lista de tarefas que a equipe deve realizar
    tasks=[buscar, redigir, editar],

    # Define se a equipe deve imprimir informações detalhadas sobre suas ações
    verbose=True
)

# Rodando o crew

In [60]:
# Define o tema do post que será criado pela equipe
tema_do_post = 'CrewAI e um caso de escrita de post no Linkedin, como é esse caso'

# Cria um dicionário com as entradas necessárias para a equipe
entradas = {
    "tema": tema_do_post  # Define o tema do post como entrada para a equipe
}

# Inicia a equipe e executa as tarefas com as entradas fornecidas
resultado = equipe.kickoff(inputs=entradas)

# Agent: Buscador de Conteúdo
## Task: 1. Identifique e priorize as últimas tendências e notícias sobre o CrewAI e um case de escrita de post no Linkedin, como é esse caso, considerando a relevância e a atualidade das informações.
2. Defina o público-alvo, considerando seus interesses, necessidades e pontos de dor, para garantir que o conteúdo seja relevante e atraente para eles.
3. Inclua palavras-chave de SEO relevantes e fontes confiáveis, para garantir que o conteúdo seja encontrado e considerado autorizado.


# Agent: Buscador de Conteúdo
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"CrewAI latest news\"}"
## Tool Output: 

Search results: Title: CrewAI Launches Multi-Agentic Platform to Deliver on the
Link: https://www.globenewswire.com/news-release/2024/10/22/2966872/0/en/CrewAI-Launches-Multi-Agentic-Platform-to-Deliver-on-the-Promise-of-Generative-AI-for-Enterprise.html
Snippet: CrewAI announces Series A funding, new enterprise offering, and 10 milli

In [61]:
from IPython.display import display, Markdown

# Se `resultado` não for uma string, converta para string
display(Markdown(str(resultado)))


🔍 Explorando a vanguarda da IA autônoma com a CrewAI 💡🚀

No mercado em rápida expansão de agentes de IA, a CrewAI emergiu como pioneira, garantindo um financiamento de $18 milhões e lançando uma plataforma inovadora de vários agentes para empresas gerenciarem autonomamente seus fluxos de trabalho complexos. 🚀📈

A CrewAI opera mais de 10 milhões de agentes mensalmente, com quase metade das empresas da Fortune 500 🏦✨ aproveitando sua estrutura open-source. E a recente estreia do CrewAI Enterprise, com adoção inicial de 150 clientes corporativos beta, promete simplificar o design, a implantação e o gerenciamento de agentes de AI para organizações de grande porte. 👥🔄⚙️

A plataforma multi-agente permite aos profissionais construir e implementar fluxos de trabalho automatizados usando qualquer modelo de linguagem grande (LLM) e plataforma em nuvem ☁️.

Os agentes da CrewAI são aplicáveis em uma ampla gama de setores, mostrando sua versatilidade:

👨‍💼 Planejamento Estratégico
💰 Finanças
📊 Métricas de Desempenho
🤖 Automação de IA
👩‍⚕️ Saúde
🧠 Aprendizado de Máquina
📈 Enriquecimento de Dados
📣 Marketing
☁️ Soluções em Nuvem
🔗 Integração de IA
📦 Cadeia de Suprimentos
⚙️ Automação de Tarefas
📚 Inteligência de Negócios
🎯 Posicionamento de Marca
💹 Otimização de Receita
🌱 Crescimento de Negócios
👩‍💼 Recursos Humanos

Para uma aprendizagem imersiva em sistemas multi-agentes, explore o curso 'Multi AI Agent Systems' de João Moura em colaboração com a DeepLearning.AI. Este curso, endossado por profissionais de IA, aplica o framework da CrewAI para decompor tarefas complexas para vários agentes de IA, aperfeiçoando técnicas como atuação, uso de ferramentas e colaboração entre agentes.

Não perca essa oportunidade de surfar a onda da IA, em direção a um ambiente de negócios mais colaborativo e eficiente. 🌊🤖💼

#CrewAI #MultiAIAgents #DeepLearningAI #AI #MachineLearning #AIFunding.